# 1. Importing Libraries ✅

In [2]:
# numpy and scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.linalg import hadamard

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F

# pandas
import pandas as pd

In [8]:
!pip freeze >> requirements.txt


# 1. Obtaining the 5G LDPC Codes

# 2. Copying the LLR ✅

In [4]:
# Define an example parity-check matrix (H)
H = torch.tensor([
    [1, 1, 0, 0],  # Check 1
    [0, 1, 1, 1],  # Check 2
    [1, 0, 0, 1]   # Check 3
], dtype=torch.float32)  # Shape: (3, 4)

# Transpose H to get H_T
H_T = H.T  # Shape: (4, 3)

# Example LLR vector (batch_size=2, num_vars=4)
LLR = torch.tensor([
    [0.0, 0.0, 0.0, 0.0],  # Batch 1
    [-0.3, 0.7, -0.9, 0.2]   # Batch 2
], dtype=torch.float32)  # Shape: (2, 4)

# Step 1: Find indices where H_T == 1
indices = (H_T == 1).nonzero(as_tuple=True)  # Returns tuple (row_indices, col_indices)

# Count number of 1s aka total number of messages
nodes = torch.sum(H == 1).item()
print(nodes)
print(indices[0]) # row indices
# print(indices[1]) # column indices
print("\n")

# Step 2: Extract LLR values using row indices (H_T rows correspond to original H columns)
# copied_LLR = LLR[:, indices[0]]  # Shape: (batch_size, num_selected_values)
# print(input_mapping_LLR[0]) # batch 1
# print(input_mapping_LLR[1]) # batch 2

# print("Copied LLR:")
# print(copied_LLR)

7
tensor([0, 0, 1, 1, 2, 3, 3])




# CL and VL index mapping ✅

In [5]:
def get_LLR_indexes(H_to_LLR_mapping_T):
    """
    Creates a 2D tensor mapping each LLR index to all other indices in the same row.

    Args:
        H_to_LLR_mapping_T (torch.Tensor): Transposed mapping matrix (num_vars, num_checks)

    Returns:
        torch.Tensor: 2D tensor where each row corresponds to an LLR index,
                      and columns contain indices of other LLRs in the same row.
                      Uses -1 for padding if needed.
    """
    num_ones = (H_to_LLR_mapping_T >= 0).sum().item()  # Number of valid LLR indices

    # Dictionary to store shared indices for each LLR index
    check_indices_dict = {i: [] for i in range(num_ones)}
    var_indices_dict = {i: [] for i in range(num_ones)}

    # Obtain Check Layer LLR Indexes
    for row in range(H_to_LLR_mapping_T.shape[0]):
        # Get valid indices (ignore -1)
        check_indices = H_to_LLR_mapping_T[row][H_to_LLR_mapping_T[row] != -1]

        # Map each LLR index to other LLR indices in the same row
        for idx in check_indices:
            check_indices_dict[idx.item()] = [j.item() for j in check_indices if j != idx]

    # Obtain Variable Layer LLR Indexes
    for row in range(H_to_LLR_mapping_T.T.shape[0]):
        # Get valid indices (ignore -1)
        var_indices = H_to_LLR_mapping_T.T[row][H_to_LLR_mapping_T.T[row] != -1]

        # Map each LLR index to other LLR indices in the same row
        for idx in var_indices:
            var_indices_dict[idx.item()] = [j.item() for j in var_indices if j != idx]

    # Convert dictionary to a padded 2D tensor
    check_max_neighbors = max(len(v) for v in check_indices_dict.values())  # Find max list length
    var_max_neighbours = max(len(v) for v in var_indices_dict.values())  # Find max list length

    check_indices_matrix = torch.full((num_ones, check_max_neighbors), -1, dtype=torch.long)  # Initialize with -1
    var_indices_matrix = torch.full((num_ones, var_max_neighbours), -1, dtype=torch.long)  # Initialize with -1

    for i, neighbors in check_indices_dict.items():
        check_indices_matrix[i, :len(neighbors)] = torch.tensor(neighbors)

    for i, neighbors in var_indices_dict.items():
        var_indices_matrix[i, :len(neighbors)] = torch.tensor(neighbors)

    return check_indices_matrix , var_indices_matrix

# Find indices where H_T == 1
row_indices, col_indices = (H_T == 1).nonzero(as_tuple=True)

# Create a mapping from (row, col) in H_T to LLR index
H_to_LLR_mapping = torch.full_like(H_T, -1, dtype=torch.long)  # Initialize with -1

# Assign LLR indices in natural order
for i, (row, col) in enumerate(zip(row_indices, col_indices)):
    H_to_LLR_mapping[row, col] = i  # Map H index to LLR index

# **Transpose the mapping matrix**
H_to_LLR_mapping_T = H_to_LLR_mapping.T  # Transpose

# Compute the shared LLR tensor
check_LLR_matrix , var_LLR_matrix = get_LLR_indexes(H_to_LLR_mapping_T)

# Print results
print("H_to_LLR_mapping_T:\n", H_to_LLR_mapping_T)
print("\nCheck LLR Index Matrix:\n", check_LLR_matrix)
print("\nVariable LLR Index Matrix:\n", var_LLR_matrix)

H_to_LLR_mapping_T:
 tensor([[ 0,  2, -1, -1],
        [-1,  3,  4,  5],
        [ 1, -1, -1,  6]])

Check LLR Index Matrix:
 tensor([[ 2, -1],
        [ 6, -1],
        [ 0, -1],
        [ 4,  5],
        [ 3,  5],
        [ 3,  4],
        [ 1, -1]])

Variable LLR Index Matrix:
 tensor([[ 1],
        [ 0],
        [ 3],
        [ 2],
        [-1],
        [ 6],
        [ 5]])


# Params for testing

In [6]:
# Example usage
batch_size = 2
num_nodes = 7
num_iterations = 2
depth_L = 2

# mapped LLR indexes for check and variable layer
check_index_tensor = check_LLR_matrix
var_index_tensor = var_LLR_matrix

# check and var tensor
print(f"\ncheck_index_tensor:\n{check_index_tensor}")
print(f"\nvar_index_tensor:\n{var_index_tensor}")

# Example Output Mapping Index
output_index_tensor = torch.tensor([[0, 0, 1, 1, 2, 3, 3]], dtype=torch.int64)
print(f"\noutput_index_tensor:\n{output_index_tensor}")


check_index_tensor:
tensor([[ 2, -1],
        [ 6, -1],
        [ 0, -1],
        [ 4,  5],
        [ 3,  5],
        [ 3,  4],
        [ 1, -1]])

var_index_tensor:
tensor([[ 1],
        [ 0],
        [ 3],
        [ 2],
        [-1],
        [ 6],
        [ 5]])

output_index_tensor:
tensor([[0, 0, 1, 1, 2, 3, 3]])


# Final Draft 2

In [7]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import deque

class LDPCDecoderResidual(torch.nn.Module):
    """
    LDPC Decoder with Check and Variable Layers, integrating Residual Connections and Output Mapping.
    Includes QPSK modulation, AWGN channel simulation, and SGD-based training.
    """
    def __init__(self, num_nodes, num_iterations, depth_L=2):
        super(LDPCDecoderResidual, self).__init__()

        self.num_nodes = num_nodes  # Number of variable nodes (messages)
        self.num_iterations = num_iterations  # Number of iterations for decoding
        self.depth_L = depth_L  # Number of past variable layers to store

        # Trainable weights for channel reliability and residual connections
        self.w_ch = torch.nn.Parameter(torch.ones(num_nodes))  # LLR weight
        self.w_res = torch.nn.Parameter(torch.ones(depth_L))  # Residual connection weights

        # FIFO queue for storing past variable layer outputs (max L layers)
        self.previous_VL_storage = deque(maxlen=depth_L)

    def check_layer_update(self, input_mapping_LLR, check_index_tensor):
        """
        Computes Min-Sum update for Check Nodes.
        Uses circulant shifts instead of full matrix operations for efficiency.
        """
        batch_size, num_vars = input_mapping_LLR.shape

        num_index_rows, num_selected_indices = check_index_tensor.shape

        valid_mask = check_index_tensor != -1  # Identify valid connections
        safe_indices = check_index_tensor.clone()
        # print(safe_indices)
        safe_indices[~valid_mask] = num_vars  # Replace invalid indices with safe values

        # expand input vector length with zero vector
        input_extended = torch.cat([input_mapping_LLR, torch.zeros((batch_size, 1), dtype=input_mapping_LLR.dtype)], dim=1)
        # print(input_extended) # correct

        input_expanded = input_extended.unsqueeze(0).expand(num_index_rows, -1, -1)
        # print(input_expanded.shape)
        # print(input_expanded)
        # print("\n")

        index_expanded = safe_indices.unsqueeze(1).expand(-1, batch_size, -1)
        # print(index_expanded.shape)
        # print(index_expanded)
        # print("\n")

        # Gather elements based on circulant shift positions
        selected_values = torch.gather(input_expanded, dim=2, index=index_expanded)
        selected_values[~valid_mask.unsqueeze(1).expand(-1, batch_size, -1)] = 0

        # Min-Sum Check Node Update
        sign_product = torch.prod(torch.sign(selected_values), dim=1)
        min_abs = torch.min(torch.abs(selected_values), dim=1).values

        min_sum_result = sign_product * min_abs
        check_layer_output = min_sum_result.reshape(batch_size, num_index_rows)
        return check_layer_output

    def variable_layer_update(self, input_mapping_LLR, check_to_variable_messages, variable_index_tensor, iteration):
        """
        Computes the Variable Node Update in LDPC Decoding using Min-Sum with Residual Connections.
        """
        batch_size, num_vars = check_to_variable_messages.shape
        num_vars_mapped, max_neighbors = variable_index_tensor.shape
        num_messages = check_to_variable_messages.shape[1]

        valid_mask = variable_index_tensor != -1
        safe_indices = variable_index_tensor.clone()
        safe_indices[~valid_mask] = num_messages

        extended_check_to_variable = torch.cat([
            check_to_variable_messages,
            torch.zeros((batch_size, 1), dtype=check_to_variable_messages.dtype)
        ], dim=1)

        check_to_variable_expanded = extended_check_to_variable.unsqueeze(0).expand(num_vars_mapped, -1, -1)
        index_expanded = safe_indices.unsqueeze(1).expand(-1, batch_size, -1)

        # Gather and sum messages
        gathered_messages = torch.gather(check_to_variable_expanded, dim=2, index=index_expanded)
        gathered_messages[~valid_mask.unsqueeze(1).expand(-1, batch_size, -1)] = 0
        summed_messages = torch.sum(gathered_messages, dim=2).T

        # Compute new variable node messages
        weighted_LLR = self.w_ch * input_mapping_LLR
        res_contrib = torch.zeros_like(input_mapping_LLR)
        for t in range(1, min(self.depth_L + 1, iteration + 1)):
            res_contrib += self.w_res[t - 1] * self.previous_VL_storage[-t]

        Q_new = summed_messages + weighted_LLR + res_contrib
        self.previous_VL_storage.append(Q_new.clone())
        return Q_new

    def output_mapping(self, final_LLR, output_index_tensor):
      """
      Output Mapping Layer: Converts LLRs to soft-bit probabilities and hard decision bits.

      Args:
          final_LLR (torch.Tensor): Log-Likelihood Ratios (LLRs) computed from the last iteration.
                                    Shape: (batch_size, num_nodes)
          output_index_tensor (torch.Tensor): Index mapping for which LLRs contribute to each output node.
                                              Shape: (num_output_nodes, num_inputs_per_output)

      Returns:
          hard_bits (torch.Tensor): Hard decision bits (0 or 1) after thresholding LLRs.
          soft_bits (torch.Tensor): Soft-bit probabilities (sigmoid applied LLRs).
      """

      # Step 1: Retrieve batch size and number of nodes from the final LLR shape
      batch_size, num_nodes = final_LLR.shape

      # Step 2: Retrieve number of output nodes and number of LLRs mapped to each output node
      num_output_nodes, num_inputs_per_output = output_index_tensor.shape
      # print(output_index_tensor) this is correct

      # Step 3: Create a mask to identify valid indices (ignore -1 entries, which represent no connection)
      valid_mask = output_index_tensor != -1

      # Step 4: Clone output index tensor and replace all -1 values with a safe index (out of range)
      safe_indices = output_index_tensor.clone()
      safe_indices[~valid_mask] = num_nodes  # Assigns invalid indices to a safe out-of-range value

      # Step 5: Extend `final_LLR` by adding an extra column of zeros
      # This allows safe gathering for out-of-range indices assigned in the previous step
      extended_LLR = torch.cat([final_LLR, torch.zeros((batch_size, 1), dtype=final_LLR.dtype)], dim=1)

      # Step 6: Expand the `extended_LLR` tensor for gathering
      # Expanding ensures that we process all output nodes simultaneously
      final_LLR_expanded = extended_LLR.unsqueeze(0).expand(num_output_nodes, -1, -1)

      # Step 7: Expand `safe_indices` to match batch size
      index_expanded = safe_indices.unsqueeze(1).expand(-1, batch_size, -1)

      # Step 8: Gather LLR values based on the index mapping
      # This extracts the LLRs that contribute to each output node
      gathered_outputs = torch.gather(final_LLR_expanded, dim=2, index=index_expanded)

      # Get unique indices
      unique_labels = torch.unique(output_index_tensor)

      # print("inputs")
      # print(gathered_outputs)
      # print("\nvalues")
      # print(output_index_tensor)

      # Group values and compute column-wise mean for each group
      grouped_means = [gathered_outputs[0][:, output_index_tensor[0] == label].mean(dim=1, keepdim=True) for label in unique_labels]

      # Stack results into shape [4, 2, 1]
      aggregated_LLR = torch.stack(grouped_means)  # Shape [4, 2, 1]

      # Remove last dimension (squeeze) and transpose to get shape [2, 4]
      aggregated_LLR = aggregated_LLR.squeeze(-1).T

      # Step 13: Compute soft-bit probabilities using the sigmoid function
      soft_bits = torch.sigmoid(aggregated_LLR)  # Converts LLRs to probabilities (range: 0 to 1)

      # Step 14: Compute hard decision bits based on LLR thresholding
      hard_bits = (aggregated_LLR <= 0).int()  # If LLR ≤ 0 → Decoded bit = 1, Else → 0

      # Step 15: Return both hard decision bits and soft-bit probabilities
      return hard_bits, soft_bits


    def forward(self, input_mapping_LLR, output_index_tensor, check_index_tensor, var_index_tensor):
        """ Full forward pass through LDPC decoding layers. """
        batch_size = input_mapping_LLR.shape[0]
        Q = torch.zeros(batch_size, self.num_nodes, self.num_iterations, device=input_mapping_LLR.device)
        print(Q)

        for l in range(self.num_iterations):
            check_messages = self.check_layer_update(Q[:, :, l - 1] if l > 0 else input_mapping_LLR, check_index_tensor)

            if l == self.num_iterations - 1:
                return self.output_mapping(check_messages, output_index_tensor)

            Q[:, :, l] = self.variable_layer_update(input_mapping_LLR, check_messages, var_index_tensor, l)
            print(Q)

def generate_zero_codewords(batch_size, num_bits):
    """
    Generates two zero codewords (before modulation).

    Args:
        num_bits (int): Number of bits per codeword.

    Returns:
        torch.Tensor: A tensor of shape (2, num_bits) representing two zero codewords.
    """
    # Create two rows of all-zero bits
    codewords = torch.zeros((batch_size, num_bits), dtype=torch.int64)
    return codewords


def qpsk_modulate(bit_sequences):
    """
    Maps a batch of bit sequences to QPSK symbols.

    Args:
        bit_sequences (torch.Tensor): Shape [batch_size, num_bits], containing {0,1}.

    Returns:
        torch.Tensor: Shape [batch_size, num_symbols] containing QPSK symbols.
    """
    batch_size, num_bits = bit_sequences.shape

    # Ensure even number of bits (pad if necessary)
    if num_bits % 2 != 0:
        bit_sequences = torch.cat([bit_sequences, torch.zeros((batch_size, 1), dtype=torch.int64)], dim=1)

    # Reshape to bit pairs for QPSK mapping
    bit_pairs = bit_sequences.view(batch_size, -1, 2)  # Shape: [batch_size, num_symbols, 2]

    # QPSK Mapping (Gray-coded)
    mapping = {
        (0, 0): complex(1, 1),   # +1 + j1
        (0, 1): complex(1, -1),  # +1 - j1
        (1, 0): complex(-1, 1),  # -1 + j1
        (1, 1): complex(-1, -1)  # -1 - j1
    }

    # Convert bit pairs to QPSK symbols
    qpsk_symbols = torch.tensor(
        [[mapping[tuple(bits.tolist())] for bits in sample] for sample in bit_pairs]
    )

    return qpsk_symbols  # Shape: [batch_size, num_symbols]

def awgn_channel(signal, snr_db):
    """
    Simulates an AWGN channel by adding noise to the input signal.

    Args:
        signal (torch.Tensor): QPSK symbols.
        snr_db (float): Signal-to-Noise Ratio in dB.

    Returns:
        torch.Tensor: Noisy received symbols.
    """
    snr_linear = 10 ** (snr_db / 10)  # Convert SNR from dB to linear scale

    # noise = noise_std * (torch.randn_like(signal) + 1j * torch.randn_like(signal))  # Generate complex noise

    # Compute noise standard deviation (corrected)
    noise_std = torch.sqrt(torch.tensor(1 / (2 * snr_linear), dtype=signal.dtype))

    # Generate correct complex Gaussian noise (independent real & imaginary parts)
    noise = noise_std * (torch.randn_like(signal.real) + 1j * torch.randn_like(signal.imag))

    # Add noise to QPSK symbols
    received_signal = signal + noise

    return received_signal

def qpsk_demodulate(received_signal, snr_db):
    """
    QPSK Demodulation: Converts received symbols to Log-Likelihood Ratios (LLRs) for multiple batches.

    Args:
        received_signal (torch.Tensor): Noisy QPSK symbols of shape (batch_size, num_symbols), complex values.
        snr_db (float): Signal-to-Noise Ratio in dB.

    Returns:
        torch.Tensor: LLR values of shape (batch_size, num_symbols * 2), where each symbol produces 2 LLRs.
    """
    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr_db / 10)

    # Compute noise variance from the linear SNR value
    noise_var = 1 / (2 * snr_linear)

    # Compute LLR values for the real part (first bit in QPSK symbol)
    llr_real = 2 * received_signal.real / noise_var  # Shape: (batch_size, num_symbols)

    # Compute LLR values for the imaginary part (second bit in QPSK symbol)
    llr_imag = 2 * received_signal.imag / noise_var  # Shape: (batch_size, num_symbols)

    # Concatenate real and imaginary LLRs along the last dimension
    llr_output = torch.cat((llr_real, llr_imag), dim=-1)  # Shape: (batch_size, num_symbols * 2)

    return llr_output

# Training function
def train_decoder(decoder, num_epochs, learning_rate, variable_bit_length):
    """
    Trains the LDPC decoder using Stochastic Gradient Descent (SGD).
    Args:
        decoder (LDPCDecoderResidual): The LDPC decoder model.
        num_epochs (int): Number of training epochs.
        learning_rate (float): Learning rate for the optimizer.

    Returns:
        None (Prints training loss at each epoch).
    """
    # Initialize the optimizer for training using SGD
    optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    # Loop over each epoch
    for epoch in range(num_epochs):

        # Generate a random SNR value (in dB) between -4 dB and 0 dB
        snr_db = torch.FloatTensor(1).uniform_(-4, 0).item()

        # # Generate an all-zero codeword (size: 2 * num_nodes because QPSK maps 2 bits per symbol)
        messages = generate_zero_codewords(batch_size, variable_bit_length)
        # print("Line 302:")
        # print(messages)

        # Modulate input LLR using QPSK
        qpsk_symbols = qpsk_modulate(messages)

        # Pass the QPSK symbols through an AWGN channel with the selected SNR
        received_signal = awgn_channel(qpsk_symbols, snr_db)

        # Demodulate the received signal to obtain LLRs
        llrs = qpsk_demodulate(received_signal, snr_db)

        # Copy LLRs depending on the H matrix observed
        copied_LLR = llrs[:, indices[0]]
        # print(copied_LLR)

        # Forward pass: Decode the received LLR values using the LDPC decoder
        hard_bits, soft_bits = decoder(copied_LLR, output_index_tensor, check_index_tensor, var_index_tensor)

        # Compute loss using Binary Cross-Entropy (BCE)
        # The target is the original transmitted bits converted to float
        target = messages.float()
        loss = F.binary_cross_entropy(soft_bits, target)

        # Zero gradients before backpropagation
        optimizer.zero_grad()

        # Backpropagate the loss
        loss.backward()

        # Update model parameters using SGD
        optimizer.step()

        # Print training progress
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item()}")
        # print(f"Input LLR: {llrs}")
        # print(f"Soft Bits:\n{soft_bits}\n")

# variable bit length
n = H.shape[1] # number of cols = number of var bits
# print(n)

# print(f"Our input:\n{input_mapping_LLR}\n")
decoder = LDPCDecoderResidual(num_nodes=7, num_iterations=3, depth_L=2)
train_decoder(decoder, num_epochs=1, learning_rate=0.01, variable_bit_length=n)

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])
tensor([[[ 2.4578,  0.0000,  0.0000],
         [ 4.9123,  0.0000,  0.0000],
         [ 5.5499,  0.0000,  0.0000],
         [ 5.1239,  0.0000,  0.0000],
         [ 4.6772,  0.0000,  0.0000],
         [ 0.6936,  0.0000,  0.0000],
         [ 0.8863,  0.0000,  0.0000]],

        [[ 3.5909,  0.0000,  0.0000],
         [ 0.7105,  0.0000,  0.0000],
         [ 4.9090,  0.0000,  0.0000],
         [ 2.0285,  0.0000,  0.0000],
         [-0.1927,  0.0000,  0.0000],
         [-0.4260,  0.0000,  0.0000],
         [ 0.7105,  0.0000,  0.0000]]], grad_fn=<CopySlices>)
tensor([[[ 2.4578,  4.9156,  0.0000],
         [ 4.9123, 12.2791,  0.0000],
         [ 5.5499, 11.0997,  0.0000],
  

# Used for testing 5G LDPC codes

In [7]:
import matplotlib.pyplot as plt
import torch

def evaluate_ldpc(H, snr_range, num_trials=100):
    """
    Evaluates BER and FER for a given LDPC parity-check matrix over an SNR range.

    Args:
        H (torch.Tensor): Parity-check matrix.
        snr_range (list): List of SNR values to test.
        num_trials (int): Number of trials per SNR.

    Returns:
        (list, list): BER and FER values for each SNR.
    """
    ber_results = []
    fer_results = []

    for snr_db in snr_range:
        total_ber = 0
        total_fer = 0

        for _ in range(num_trials):
            # Generate all-zero codeword
            transmitted_bits = torch.zeros((1, H.shape[1]), dtype=torch.int64)

            # Modulate using QPSK
            qpsk_symbols = qpsk_modulate(transmitted_bits.view(-1))

            # Pass through AWGN channel
            received_signal = awgn_channel(qpsk_symbols, snr_db)

            # Demodulate to LLRs
            llrs = qpsk_demodulate(received_signal, snr_db)

            # Decode using LDPC
            decoded_bits, _ = decoder(llrs, output_index_tensor, check_index_tensor, var_index_tensor)

            # Compute BER and FER
            ber, fer = compute_ber_fer(transmitted_bits, decoded_bits)
            total_ber += ber
            total_fer += fer

        ber_results.append(total_ber / num_trials)
        fer_results.append(total_fer / num_trials)

    return ber_results, fer_results

# Define SNR range
snr_range = list(range(-4, 6))

# Long & Short 5G LDPC Codes
with open('/content/NR_2_0_4.txt', 'r') as f:
    lines = f.readlines()

# Parse each line into a list of numbers
H_short = []
for line in lines:
    # Split on whitespace and convert each piece to float (or int)
    row = [float(x) for x in line.split()]
    H_short.append(row)
H_short = torch.tensor(H_short)
print(H_short.shape)
print(H_short)

# H_long =

# Run simulations for both short and long LDPC codes
ber_short, fer_short = evaluate_ldpc(H_short, snr_range)
ber_long, fer_long = evaluate_ldpc(H_long, snr_range)

# Plot BER vs. SNR
plt.plot(snr_range, ber_short, label="Short LDPC")
plt.plot(snr_range, ber_long, label="Long LDPC")
plt.xlabel("SNR (dB)")
plt.ylabel("BER")
plt.legend()
plt.title("BER vs. SNR")
plt.show()

torch.Size([42, 52])
tensor([[ 1.,  1.,  0.,  ..., -1., -1., -1.],
        [ 3., -1., -1.,  ..., -1., -1., -1.],
        [ 1.,  2., -1.,  ..., -1., -1., -1.],
        ...,
        [ 3., -1., -1.,  ...,  0., -1., -1.],
        [-1., -1.,  0.,  ..., -1.,  0., -1.],
        [-1.,  1., -1.,  ..., -1., -1.,  0.]])


NameError: name 'qpsk_modulate' is not defined

# Testing the base graph extension

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# -------------------------
# 1) DEFINE BASE GRAPH
# -------------------------
# Let's say we have a small base graph with 2 rows (check nodes) and 3 columns (variable nodes).
# SHIFT[r,c] = -1 means NO edges; otherwise SHIFT[r,c] = an integer shift.
# We choose a small example for demonstration:

BG = [
    [ 0,  1, -1],
    [-1,  0,  2]
]
R_BG = len(BG)       # 2
C_BG = len(BG[0])    # 3

# Lifting factor
Z = 2  # So final parity-check matrix is (2*Z) x (3*Z) = 4 x 6

# -------------------------
# 2) NEURAL MODULE FOR PARAMETER TYING
# -------------------------
# We'll create a simple "per-base-graph-cell" neural transform (a single Linear layer)
# that we will apply to the messages for edges coming from that cell.

class CellTransform(nn.Module):
    def __init__(self, in_features=1, out_features=1):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features, bias=True)

    def forward(self, x):
        # x shape: (any_batch, in_features)
        return self.linear(x)

class TiedNeuralLDPCDecoder(nn.Module):
    def __init__(self, base_graph, Z):
        super().__init__()
        self.base_graph = base_graph
        self.Z = Z

        # Build a small nn.ModuleDict for each cell that has shift >= 0
        # Key: (r,c)
        # Value: a CellTransform (or could be a small MLP)
        self.cell_modules = nn.ModuleDict()

        for r in range(len(base_graph)):
            for c in range(len(base_graph[0])):
                if base_graph[r][c] >= 0:
                    key = f"({r},{c})"
                    self.cell_modules[key] = CellTransform(in_features=1, out_features=1)

    def forward(self, channel_llrs):
        """
        channel_llrs: a tensor of shape [C_BG * Z], i.e. for each variable node in the expanded domain,
                      we have 1 LLR. (This simple example uses 1D for demonstration.)

        We'll do ONE iteration of (check-update -> variable-update) in a naive way.
        """
        # --------------------------------
        #  CHECK NODE UPDATE (Min-Sum-ish)
        # --------------------------------
        # We'll store check->variable messages in a tensor of same shape [C_BG*Z],
        # but we must gather them for each check node (r,z_r) -> variable node (c,z_c).
        # For simplicity, let's store a dictionary: key=(r,z_r,c,z_c), value=message
        M_c_to_v = {}

        # For each check node in expanded domain: (r in [0..R_BG-1], z_r in [0..Z-1])
        for r in range(len(self.base_graph)):
            for z_r in range(self.Z):

                # Collect all edges from (r,z_r) => (c,z_c), if SHIFT[r,c] != -1
                # Condition: z_r = (z_c + shift[r][c]) mod Z
                for c in range(len(self.base_graph[r])):
                    shift_val = self.base_graph[r][c]
                    if shift_val >= 0:
                        # possible edges
                        for z_c in range(self.Z):
                            if z_r == (z_c + shift_val) % self.Z:
                                # This means check node (r,z_r) connects to variable node (c,z_c).
                                # We need the incoming variable->check message. In this simple example,
                                # let's approximate the variable->check message as the channel_llr
                                # from that variable node (no prior iteration).

                                # channel_llrs index is c*Z + z_c
                                v2c_message = channel_llrs[c*self.Z + z_c].view(1,1)  # shape [1,1]

                                # Pass through the "check transform" for this cell = (r,c).
                                # Or do a simple "min-sum" style. We'll illustrate a trivial transform:
                                key_module = f"({r},{c})"
                                module = self.cell_modules[key_module]
                                c2v_message = module(v2c_message)  # shape [1,1]

                                # For a real check node update, you'd combine messages from ALL other variable nodes,
                                # but let's keep it very minimal.

                                M_c_to_v[(r,z_r,c,z_c)] = c2v_message
                    else:
                        # shift_val == -1 => no edges
                        pass

        # --------------------------------
        #  VARIABLE NODE UPDATE
        # --------------------------------
        # Similarly, each variable node (c,z_c) will combine incoming check->var messages from all checks
        # that connect to it. We'll do a naive sum + channel prior, then a small transform.

        # We'll store new variable->check messages. For demonstration, let's just produce final LLRs.
        final_llrs = torch.zeros_like(channel_llrs)

        for c in range(C_BG):
            for z_c in range(self.Z):
                # Identify which checks (r,z_r) connect here:
                # Condition: z_r = (z_c + shift[r,c]) mod Z
                # We'll sum up all M_c_to_v[r,z_r,c,z_c] that exist.

                sum_msg = channel_llrs[c*self.Z + z_c].clone()  # start with channel prior
                # We'll do a naive sum of all check->var messages for demonstration
                for r in range(R_BG):
                    shift_val = self.base_graph[r][c]
                    if shift_val >= 0:
                        # find z_r that connects:
                        z_r = (z_c + shift_val) % self.Z
                        if (r,z_r,c,z_c) in M_c_to_v:
                            sum_msg += M_c_to_v[(r,z_r,c,z_c)].squeeze()  # shape -> scalar
                final_llrs[c*self.Z + z_c] = sum_msg

        return final_llrs

# -------------------------
# 3) DEMO USAGE
# -------------------------

if __name__ == "__main__":
    # Create our model
    decoder = TiedNeuralLDPCDecoder(BG, Z)

    # Example channel LLRs for the expanded code (C_BG*Z = 3*2=6 variables)
    # Let's say we have some random LLRs:
    channel_llrs = torch.tensor([ 0.8, -1.2, 1.5, 0.2, -0.7, 0.9 ], dtype=torch.float)

    # Forward pass (one iteration)
    final_llrs = decoder(channel_llrs)
    print("Final LLRs after 1 iteration:", final_llrs.detach().numpy())

    # Example of a training step:
    # Suppose we have some ground-truth bits we want to recover.
    # We'll do a simple supervised loss (e.g., MSE on LLR vs. desired sign).
    # This is purely illustrative.

    optimizer = torch.optim.Adam(decoder.parameters(), lr=1e-3)
    target_bits = torch.tensor([0,1,0,0,1,1], dtype=torch.float)  # 0 or 1
    # Convert bits to "ideal" LLR sign: + for bit=0, - for bit=1
    desired_llrs = 5.0*(1.0 - 2.0*target_bits)  #  bit0 => +5.0, bit1 => -5.0

    for step in range(10):
        optimizer.zero_grad()
        out_llrs = decoder(channel_llrs)
        loss = F.mse_loss(out_llrs, desired_llrs)
        loss.backward()
        optimizer.step()
        print(f"Step {step}, Loss={loss.item():.4f}")


Final LLRs after 1 iteration: [ 0.01361436 -1.217307    1.8415205  -0.84667087 -0.5372044   1.7802583 ]
Step 0, Loss=24.8701
Step 1, Loss=24.8522
Step 2, Loss=24.8343
Step 3, Loss=24.8164
Step 4, Loss=24.7985
Step 5, Loss=24.7807
Step 6, Loss=24.7628
Step 7, Loss=24.7450
Step 8, Loss=24.7272
Step 9, Loss=24.7093


# 5G LDPC CODES, can use ✅
https://github.com/manuts/NR-LDPC-BG

# 1.1 Find Long-length 5G LDPC code 🔴 (finish by 21 Feb)

In [ ]:
# Define Base Matrix 1 (BM1) - 46x68 (Simplified Example with Random Values)
BM1 = torch.randint(-1, 4, (46, 68))  # -1 represents zero block, 0-3 are circulant shifts

# Define Base Matrix 2 (BM2) - 42x52 (Simplified Example with Random Values)
BM2 = torch.randint(-1, 4, (42, 52))  # -1 represents zero block, 0-3 are circulant shifts

# Define Lifting Factors (Z) for different scenarios
Z_values = [16, 32, 64, 128, 256]  # 5G NR standard Z values

# Function to expand base matrix using a selected Z value
def expand_base_matrix(base_matrix, Z):
    """
    Expands the given base matrix by replacing each entry with a ZxZ block.
    - A zero block (-1 in base matrix) becomes a ZxZ zero matrix.
    - A circulant shift value (0, 1, 2, 3) becomes a shifted identity matrix.
    """
    rows, cols = base_matrix.shape
    H_expanded = torch.zeros((rows * Z, cols * Z), dtype=torch.float32)

    for i in range(rows):
        for j in range(cols):
            shift = base_matrix[i, j].item()
            if shift == -1:
                # Zero Block (ZxZ Zero Matrix)
                H_expanded[i * Z:(i + 1) * Z, j * Z:(j + 1) * Z] = torch.zeros((Z, Z))
            else:
                # Circulant Shifted Identity Matrix
                I_Z = torch.eye(Z)  # Identity Matrix of size ZxZ
                I_Z = torch.roll(I_Z, shifts=shift, dims=1)  # Shift columns by "shift" value
                H_expanded[i * Z:(i + 1) * Z, j * Z:(j + 1) * Z] = I_Z

    return H_expanded

# Example: Expanding BM1 with Z=64
Z_selected = 64  # Example Z value
H_expanded_BM1 = expand_base_matrix(BM1, Z_selected)  # Full Parity-Check Matrix for BM1
H_expanded_BM2 = expand_base_matrix(BM2, Z_selected)  # Full Parity-Check Matrix for BM2

# Display results
df_BM1 = pd.DataFrame(BM1.numpy())
df_BM2 = pd.DataFrame(BM2.numpy())

print(df_BM1)
print(df_BM2)

    0   1   2   3   4   5   6   7   8   9   ...  58  59  60  61  62  63  64  \
0    1  -1   2   1  -1  -1   1   2   1   0  ...  -1   1  -1   3  -1  -1  -1   
1    0   2   2   3  -1   2  -1   0   3   3  ...   2   0   1  -1   2   1  -1   
2    0  -1   2   2   1   1  -1  -1   1   1  ...   2   3   0  -1   1   0  -1   
3   -1  -1   1   1   2   2   3   2   2   1  ...   0   1   2   3   0   3   3   
4    2  -1  -1  -1   1   0   0   1   1  -1  ...  -1   2   0  -1   0   3   1   
5   -1   1   0  -1  -1   2   1   0   2  -1  ...   0   1   2   1   3  -1   3   
6    2   1   1   1   0   0   0   1   0  -1  ...   1   1   1   0   2   0   0   
7   -1   0   3   3   0   0   2   1  -1   0  ...  -1   0   2   3   0   1   3   
8   -1   1   3   0   2   2  -1   3   0   2  ...   0   1  -1  -1   3   0  -1   
9    0   0   3   2   3  -1   3   3   1   3  ...   2   3   3   3   2   0   2   
10   0   1   2   2   3  -1  -1  -1   1   0  ...   1   3   2   0   1   3   1   
11   2   3   1   2   1   2  -1  -1  -1  -1  ...   1 

# 1.2 Short-length 5G LDPC code 🔴 (finish by 21 Feb)

#Check Layer ✅

In [ ]:
def min_sum_operation_reordered(input_tensor, index_tensor):
    """
    For each row in input_tensor, apply every row in index_tensor, collect results,
    and apply the Min-Sum operation along columns, with batch results stacked together.

    Handles -1 indices by replacing them with `num_vars` and extending input_tensor with extra zeros.

    Args:
        input_tensor (torch.Tensor): Shape (batch_size, num_vars)
        index_tensor (torch.Tensor): Shape (num_index_rows, num_selected_indices)

    Returns:
        torch.Tensor: Min-Sum results after applying index_tensor to input_tensor.
                      Shape: (batch_size * num_index_rows, num_selected_indices)
    """
    batch_size, num_vars = input_tensor.shape  # (B, V)
    num_index_rows, num_selected_indices = index_tensor.shape  # (I, S)

    # Step 1: Create a mask for valid indices (ignoring -1)
    valid_mask = index_tensor != -1

    # Step 2: Replace -1 with a safe dummy index (`num_vars`)
    safe_indices = index_tensor.clone()
    safe_indices[~valid_mask] = num_vars  # Safe index (out of range)

    # Step 3: Extend input_tensor with an extra column of zeros
    input_extended = torch.cat([input_tensor, torch.zeros((batch_size, 1), dtype=input_tensor.dtype)], dim=1)
    # Shape: (batch_size, num_vars + 1)

    # Step 4: Expand input tensor to apply all index_tensor rows
    input_expanded = input_extended.unsqueeze(0).expand(num_index_rows, -1, -1)
    # Shape: (num_index_rows, batch_size, num_vars + 1)

    # Step 5: Expand index tensor to match batch dimension
    index_expanded = safe_indices.unsqueeze(1).expand(-1, batch_size, -1)
    # Shape: (num_index_rows, batch_size, num_selected_indices)

    # Step 6: Gather elements
    selected_values = torch.gather(input_expanded, dim=2, index=index_expanded)
    # Shape: (num_index_rows, batch_size, num_selected_indices)

    # Step 7: Apply mask: Set invalid gathered values (from -1) to zero
    selected_values[~valid_mask.unsqueeze(1).expand(-1, batch_size, -1)] = 0

    # Step 8: Rearrange `selected_values` to have all batch 1 results first
    selected_values_reordered = selected_values.permute(1, 0, 2).reshape(batch_size * num_index_rows, num_selected_indices)
    # Shape: (batch_size * num_index_rows, num_selected_indices)

    # Step 9: Apply Min-Sum
    sign_product = torch.prod(torch.sign(selected_values_reordered), dim=1)  # Sign product
    min_abs = torch.min(torch.abs(selected_values_reordered), dim=1).values  # Min absolute value

    # Step 10: Compute Min-Sum result
    min_sum_result = sign_product * min_abs  # Element-wise multiplication
    # Shape: (batch_size * num_index_rows,)

    return selected_values_reordered, min_sum_result.reshape(batch_size, num_index_rows)

# Example input message tensor (batch_size=2, num_vars=7)
input_tensor = torch.tensor([
    [0.2, -0.5, 0.8, -0.1, 0.2, 0.3, 0.1],  # Batch 1
    [-0.3, 0.7, -0.9, 0.2, 0.3, 0.2, 0.1]   # Batch 2
], dtype=torch.float32)  # Shape: (2, 7)

# Check LLR Index Matrix:
#  tensor([[ 2, -1],
#         [ 6, -1],
#         [ 0, -1],
#         [ 4,  5],
#         [ 3,  5],
#         [ 3,  4],
#         [ 1, -1]])

# Compute Min-Sum with reordered results
selected_values_reordered, check_layer_output = min_sum_operation_reordered(input_tensor, check_LLR_matrix)

print("Reordered Selected Values:\n", selected_values_reordered)
print("\nMin-Sum Result:\n", check_layer_output)


Reordered Selected Values:
 tensor([[ 0.8000,  0.0000],
        [ 0.1000,  0.0000],
        [ 0.2000,  0.0000],
        [ 0.2000,  0.3000],
        [-0.1000,  0.3000],
        [-0.1000,  0.2000],
        [-0.5000,  0.0000],
        [-0.9000,  0.0000],
        [ 0.1000,  0.0000],
        [-0.3000,  0.0000],
        [ 0.3000,  0.2000],
        [ 0.2000,  0.2000],
        [ 0.2000,  0.3000],
        [ 0.7000,  0.0000]])

Min-Sum Result:
 tensor([[ 0.0000,  0.0000,  0.0000,  0.2000, -0.1000, -0.1000, -0.0000],
        [-0.0000,  0.0000, -0.0000,  0.2000,  0.2000,  0.2000,  0.0000]])


# Variable Layer ✅

In [ ]:
def variable_layer_update(input_mapping_LLR, check_to_variable_messages, variable_index_tensor):
    """
    Performs the Variable Node Update in LDPC Decoding using Min-Sum.

    Args:
        LLR (torch.Tensor): Initial log-likelihood ratios (batch_size, num_vars).
        check_to_variable_messages (torch.Tensor): Messages from check nodes (batch_size, num_messages).
        variable_index_tensor (torch.Tensor): Indices of check node messages that share the same column.

    Returns:
        torch.Tensor: Updated variable node messages (batch_size, num_vars).
    """
    batch_size, num_vars = check_to_variable_messages.shape  # (B, V)
    num_vars_mapped, max_neighbors = variable_index_tensor.shape  # (V', S)
    num_messages = check_to_variable_messages.shape[1]  # Number of check node messages

    # Create a mask where valid indices are True (ignoring -1 values)
    valid_mask = variable_index_tensor != -1

    # Replace -1 with a safe dummy index (num_messages) which we will zero out later
    safe_indices = variable_index_tensor.clone()
    safe_indices[~valid_mask] = num_messages  # Use an out-of-range index***

    # Extend check_to_variable_messages with a row of zeros at index num_messages
    extended_check_to_variable = torch.cat([
        check_to_variable_messages,
        torch.zeros((batch_size, 1), dtype=check_to_variable_messages.dtype)
    ], dim=1)  # Shape: (batch_size, num_messages + 1)

    # Expand check-to-variable messages and indices
    check_to_variable_expanded = extended_check_to_variable.unsqueeze(0).expand(num_vars_mapped, -1, -1)
    index_expanded = safe_indices.unsqueeze(1).expand(-1, batch_size, -1)

    # Gather messages using safe indices
    gathered_messages = torch.gather(check_to_variable_expanded, dim=2, index=index_expanded)

    # Apply mask: Set invalid gathered values to 0
    gathered_messages[~valid_mask.unsqueeze(1).expand(-1, batch_size, -1)] = 0

    # Sum valid messages
    summed_messages = torch.sum(gathered_messages, dim=2)  # (num_vars_mapped, batch_size)

    # Transpose to match (batch_size, num_vars)
    summed_messages = summed_messages.T  # (batch_size, num_vars_mapped)

    # Compute updated variable messages
    variable_messages = input_mapping_LLR + summed_messages  # Add the LLR to the sum of messages

    return variable_messages

# Example Check-to-Variable Messages (batch_size=2, num_messages=3)
var_input_tensor = check_layer_output

# Compute Variable Node Messages
variable_messages = variable_layer_update(input_mapping_LLR, var_input_tensor, var_LLR_matrix)

print("Updated Variable Messages:\n", variable_messages)

Updated Variable Messages:
 tensor([[ 0.0000,  0.0000,  0.2000,  0.0000,  0.0000,  0.0000, -0.1000],
        [-0.3000, -0.3000,  0.9000,  0.7000, -0.9000,  0.2000,  0.4000]])


# Residual Connections ✅

In [ ]:
# params
batch_size = 2
num_nodes = nodes
depth_L = 2

class LDPCDecoderResidual(torch.nn.Module):
    """
    Neural LDPC decoder with residual connections.
    Implements Equation (4) from the paper.
    """
    def __init__(self, num_nodes, depth_L=2):
        super(LDPCDecoderResidual, self).__init__()

        self.num_nodes = num_nodes  # Number of variable nodes
        self.depth_L = depth_L  # Depth of residual connections

        # Trainable weights
        self.w_ch = torch.nn.Parameter(torch.ones(num_variables))  # w^{ch} weights
        self.w_res = torch.nn.Parameter(torch.ones(depth_L))  # w^{(l-t)} weights for residual connections

    def forward(self, LLR, check_messages):
        """
        Compute variable node update with residual connections.

        Args:
            LLR (torch.Tensor): Initial LLR values (batch_size, num_variables)
            check_messages (torch.Tensor): Incoming check node messages (batch_size, num_variables)

        Returns:
            Q (torch.Tensor): Updated variable node messages (batch_size, num_variables, num_iterations)
        """
        batch_size = LLR.shape[0]

        # Initialize message tensor
        Q = torch.zeros(batch_size, self.num_nodes, self.num_iterations, device=LLR.device)

        # for l in range(self.num_iterations):
        # Compute weighted channel LLR contribution
        weighted_LLR = self.w_ch * LLR

        # Compute check node contribution
        check_contrib = check_messages  # Shape: (batch_size, num_variables)

        # Residual connection contribution from previous iterations
        res_contrib = torch.zeros_like(LLR)
        for t in range(1, min(self.depth_L + 1, l + 1)):  # Ensure only past iterations are used
            res_contrib += self.w_res[t - 1] * Q[:, :, l - t]

        # Final update equation (Equation 4)
        Q[:, :, l] = weighted_LLR + check_contrib + res_contrib

        return Q  # Shape: (batch_size, num_variables, num_iterations)



# num_variables = H.shape  # Example: 10 variable nodes
# num_check_nodes = 5  # Example: 5 check nodes

# residual_input or something that stores the previous L Variable layers

# Initialize decoder
decoder = LDPCDecoderResidual(num_nodes, depth_L)

# Run decoder
Q_output = decoder(residual_input, check_layer_output)

# Output Layer ✅

In [ ]:
import torch
import torch.nn.functional as F

class LDPCDecoderOutputLayer(torch.nn.Module):
    """
    Output mapping layer for LDPC neural decoder.
    Maps the final LLRs to soft-bit values using Sigmoid and applies Cross-Entropy loss.
    """
    def __init__(self):
        super(LDPCDecoderOutputLayer, self).__init__()

    def forward(self, final_LLR, ground_truth, input_mapping_LLR):
        """
        Computes the soft-bit values and cross-entropy loss.

        Args:
            final_LLR (torch.Tensor): Output LLR vector from the last variable layer. Shape: (batch_size, num_variables).
            ground_truth (torch.Tensor): True transmitted bits (0 or 1). Shape: (batch_size, num_variables).

        Returns:
            loss (torch.Tensor): Cross-entropy loss.
            predicted_bits (torch.Tensor): Decoded soft-bit values.
        """

        result = final_LLR + input_mapping_LLR

        # Step 1: Apply Sigmoid to final LLR (σ(L_o))
        soft_bits = torch.sigmoid(final_LLR)  # Maps LLRs to probability values (0 to 1)

        # Step 2: Compute Cross-Entropy loss
        loss = F.binary_cross_entropy(soft_bits, ground_truth, reduction='none')  # Cross-entropy per bit
        print("Cross-Entropy Loss:\n", loss)

        # Step 3: Apply max function over the loss vector (for FER minimization)
        max_loss = torch.max(loss, dim=1).values  # Maximum cross-entropy value per batch

        return max_loss, soft_bits

# # Example Usage
batch_size = 2
num_variables = 7  # Number of message nodes

# # Set the random seed for reproducibility
torch.manual_seed(23)

# Simulated LLR output from the final variable layer
final_LLR = check_layer_output

# Simulated ground truth bits (binary values: 0 or 1)
ground_truth = torch.randint(0, 2, (batch_size, num_variables)).float()

# Initialize and run output mapping
output_layer = LDPCDecoderOutputLayer()
max_loss, predicted_bits = output_layer(final_LLR, ground_truth, input_mapping_LLR)

hard_bit_results = (predicted_bits > 0.5).float()

# Print results
print("\nMax Cross-Entropy Loss Per Batch:\n", max_loss)
print("\nFinal LLRs:\n", final_LLR)
print("\nSoft-Bit Values (After applying Sigmoid to LLR):\n", predicted_bits)
print("\nHard-Bit Values:\n", hard_bit_results)

Cross-Entropy Loss:
 tensor([[0.6931, 0.6931, 0.6931, 0.5981, 0.6444, 0.6444, 0.6931],
        [0.6931, 0.6931, 0.6931, 0.5981, 0.7981, 0.5981, 0.6931]])

Max Cross-Entropy Loss Per Batch:
 tensor([0.6931, 0.7981])

Final LLRs:
 tensor([[ 0.0000,  0.0000,  0.0000,  0.2000, -0.1000, -0.1000, -0.0000],
        [-0.0000,  0.0000, -0.0000,  0.2000,  0.2000,  0.2000,  0.0000]])

Soft-Bit Values (After applying Sigmoid to LLR):
 tensor([[0.5000, 0.5000, 0.5000, 0.5498, 0.4750, 0.4750, 0.5000],
        [0.5000, 0.5000, 0.5000, 0.5498, 0.5498, 0.5498, 0.5000]])

Hard-Bit Values:
 tensor([[0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0.]])


# To-Dos

1.   Output Mapping : what operation is used on the output mapping layer
2.   Test the whole network with 1 iteration, which inlcudes 1x CL and VL
3.   Check to see if we need to do anything to use cyclic property of H matrix
4.   Check if there is a threshold for calculating FER? i.e If errors per frame > x, then count as one whole frame is defective



# Obtaining LLR